# Setup

In [1]:
from elasticsearch import (
    Elasticsearch,
    helpers
)
import pickle

In [2]:
#initialize
es = Elasticsearch()

In [3]:
movies=pickle.load(open("../movies.p","rb"))

In [4]:
try:
    es.indices.delete("tmdb")
except:
    pass

#create index here?
# genres.name needs to be keyword tokenized so that 'science fiction' doesn't get split on white space
# maybe create a text field with title and overview to search against
body = {
    "mappings": {
      "movie": {
        "properties": {
          "genres": {
            "properties": {
              "name": { 
                "type": "string",
                "index": "not_analyzed"}}},
          "title": {
            "type": "string",
            "analyzer": "english"}}}}}
es.indices.create("tmdb",body=body)

{u'acknowledged': True}

In [10]:
#doc indexer
print movies

def format_doc(doc):
    action = {
        "_index": "tmdb",
        "_type": "movie",
        "_id": doc['id\r'],
        "_source": doc
        }
    return action

def index_movies():
    actions = (format_doc(doc) for doc in movies)
    results = [details for success,details in helpers.streaming_bulk(es, actions) if not success]
    return results

[{u'revenue\r': 1518594910, u'production_companies\r': [{u'name\r': u'Paramount Pictures\r', u'id\r': 4}, {u'name\r': u'Marvel Studios\r', u'id\r': 420}], u'tagline\r': u'Some assembly required.\r', u'id\r': 24428, u'vote_count\r': 6535, u'homepage\r': u'http://marvel.com/avengers_movie/\r', u'original_title\r': u'The Avengers\r', u'production_countries\r': [{u'iso_3166_1\r': u'US\r', u'name\r': u'United States of America\r'}], u'release_date\r': u'2012-05-04\r', u'budget\r': 220000000, u'overview\r': u'When an unexpected enemy emerges and threatens global safety and security, Nick Fury, director of the international peacekeeping agency known as S.H.I.E.L.D., finds himself in need of a team to pull the world back from the brink of disaster. Spanning the globe, a daring recruitment effort begins!\r', u'status\r': u'Released\r', u'belongs_to_collection\r': {u'id\r': 86311, u'backdrop_path\r': u'/zuW6fOiusv4X9nnW3paHGfXcSll.jpg\r', u'poster_path\r': u'/qJawKUQcIBha507UahUlX0keOT7.jpg\r', 

In [11]:
results = index_movies()

# Examples

In [12]:
# basic facet example
body = {
    "aggs": {
        "genres": {
            "terms": {
                "field": "genres.name"
            }
        },
    }
}
es.search(index="tmdb",body=body,size=0)

{u'_shards': {u'failed': 0, u'successful': 5, u'total': 5},
 u'aggregations': {u'genres': {u'buckets': [],
   u'doc_count_error_upper_bound': 0,
   u'sum_other_doc_count': 0}},
 u'hits': {u'hits': [], u'max_score': 0.0, u'total': 400},
 u'timed_out': False,
 u'took': 57}

In [13]:
# facet with a filter in place - notice the different aggregation numbers
body = {
    "fields":["title"],
    "query": {
        "bool": {
            "filter": [
              {"term": {"genres.name": "Science Fiction"}}
            ]
        }
    },
    "aggs": {
        "genres": {
            "terms": {
                "field": "genres.name"
            }
        },
    }

}
es.search(index="tmdb",body=body,size=5)

{u'_shards': {u'failed': 0, u'successful': 5, u'total': 5},
 u'aggregations': {u'genres': {u'buckets': [],
   u'doc_count_error_upper_bound': 0,
   u'sum_other_doc_count': 0}},
 u'hits': {u'hits': [], u'max_score': None, u'total': 0},
 u'timed_out': False,
 u'took': 21}

# Scratch

In [14]:
# see how the genre doc conuts tum to 209 while the original language counts sum to 90 (the num docs)
body = {
    "fields":["title"],
    "query":{
        "match":{
            "genres.name": "Science Fiction"}},
    "aggs": {
        "genres": {
            "terms": {
                "field": "genres.name"
            }
        },
    }

}
es.search(index="tmdb",body=body,size=100)

{u'_shards': {u'failed': 0, u'successful': 5, u'total': 5},
 u'aggregations': {u'genres': {u'buckets': [],
   u'doc_count_error_upper_bound': 0,
   u'sum_other_doc_count': 0}},
 u'hits': {u'hits': [], u'max_score': None, u'total': 0},
 u'timed_out': False,
 u'took': 8}

In [15]:
# facet with a filter in place
body = {
    "fields":["title"],
    "query": {
        "filtered": {
            "query": {
                "match":{
                    "title": "star trek"}},
            "filter": {
                "term": {
                    "genres.name": "Science Fiction"}}}},
    "aggs": {
        "genres": {
            "terms": {
                "field": "genres.name"
            }
        },
    }

}
es.search(index="tmdb",body=body,size=100)

{u'_shards': {u'failed': 0, u'successful': 5, u'total': 5},
 u'aggregations': {u'genres': {u'buckets': [],
   u'doc_count_error_upper_bound': 0,
   u'sum_other_doc_count': 0}},
 u'hits': {u'hits': [], u'max_score': None, u'total': 0},
 u'timed_out': False,
 u'took': 11}

#Trash

In [16]:
{ "query": {
    "filtered": {
      "filter": {
        "term": {
          "genres.name": "Science Fiction"}}}},
  "aggs": {
    "genres": {
      "terms": {
        "field": "genres.name"}}}}


{'aggs': {'genres': {'terms': {'field': 'genres.name'}}},
 'query': {'filtered': {'filter': {'term': {'genres.name': 'Science Fiction'}}}}}

In [17]:
x= [
    {'doc_count': 7546, 'key': 'Drama'},
    {'doc_count': 5342, 'key': 'Comedy'},
    {'doc_count': 3878, 'key': 'Thriller'},
    {'doc_count': 3753, 'key': 'Action'},
    {'doc_count': 2623, 'key': 'Romance'},
    {'doc_count': 2165, 'key': 'Adventure'},
    {'doc_count': 1981, 'key': 'Horror'},
    {'doc_count': 1861, 'key': 'Crime'},
    {'doc_count': 1640, 'key': 'Family'},
    {'doc_count': 1597, 'key': 'Science Fiction'}]
sum([y['doc_count'] for y in x])

32386

In [18]:
es.cluster.stats()['indices'].keys()

[u'count',
 u'completion',
 u'fielddata',
 u'docs',
 u'segments',
 u'shards',
 u'query_cache',
 u'percolate',
 u'store']